In [18]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import pickle


In [4]:
# read in the top 5 leagues data
pl = pd.read_csv('/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/eng-premier_league_schedule_updated.csv')
ita = pd.read_csv('/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/ita-serie_a_schedule_updated.csv')
esp = pd.read_csv('/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/esp-la_liga_schedule_updated.csv')
de = pd.read_csv('/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/ger-bundesliga_schedule_updated.csv')
fr = pd.read_csv('/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/fra-ligue_1_schedule_updated.csv')

In [22]:


# Concatenate all leagues into a single dataframe
leagues = pd.concat([pl, ita, esp, de, fr], ignore_index=True)

# Normalize score text (keep all rows)
leagues['score'] = leagues['score'].astype(str).str.strip()
leagues['score'] = leagues['score'].str.replace('[–—−]', '-', regex=True)

# Parse valid scores; keep invalid rows and mark them
valid_score = leagues['score'].str.fullmatch(r'\s*\d+\s*-\s*\d+\s*', na=False)
parsed = leagues['score'].str.extract(r'^\s*(\d+)\s*-\s*(\d+)\s*$')

# Build goal columns (nullable Int types to allow missing)
leagues['home_goals'] = pd.to_numeric(parsed[0], errors='coerce').astype('Int64')
leagues['away_goals'] = pd.to_numeric(parsed[1], errors='coerce').astype('Int64')

# Derived columns only where valid; otherwise remain NA
leagues['goal_diff'] = (leagues['home_goals'] - leagues['away_goals']).astype('Int64')

# Result: 1 home win, 0 draw, -1 away win (nullable Int)
leagues['result'] = pd.Series(pd.NA, index=leagues.index, dtype='Int8')
leagues.loc[valid_score & (leagues['goal_diff'] > 0), 'result'] = 1
leagues.loc[valid_score & (leagues['goal_diff'] == 0), 'result'] = 0
leagues.loc[valid_score & (leagues['goal_diff'] < 0), 'result'] = -1

# Optional flag to easily filter later
leagues['is_valid_score'] = valid_score

# Date as datetime
leagues['date'] = pd.to_datetime(leagues['date'], errors='coerce')

# Save without dropping any rows
leagues.to_pickle('/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/top5_leagues_schedule.pkl')

In [23]:
leagues

,week,day,date,time,home_team,score,away_team,attendance,venue,referee,...,notes,home_xg,away_xg,game_id,round,home_goals,away_goals,goal_diff,result,is_valid_score
0,1.0,Sat,2005-08-13,NaN,Aston Villa,2-2,Bolton,33263.0,Villa Park,Mike Riley,...,NaN,NaN,NaN,320758d6,NaN,2,2,0,0,True
1,1.0,Sat,2005-08-13,NaN,Everton,0-2,Manchester Utd,38607.0,Goodison Park,Graham Poll,...,NaN,NaN,NaN,8a50535d,NaN,0,2,-2,-1,True
2,1.0,Sat,2005-08-13,NaN,Fulham,0-0,Birmingham City,16550.0,Craven Cottage,Rob Styles,...,NaN,NaN,NaN,db5d09d8,NaN,0,0,0,0,True
3,1.0,Sat,2005-08-13,NaN,Manchester City,0-0,West Brom,42983.0,Etihad Stadium,Chris Foy,...,NaN,NaN,NaN,67bd8f9e,NaN,0,0,0,0,True
4,1.0,Sat,2005-08-13,NaN,Middlesbrough,0-0,Liverpool,31908.0,Riverside Stadium,Mark Halsey,...,NaN,NaN,NaN,f45106e3,NaN,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38156,34.0,Sat,2026-05-16,NaN,Lorient,nan,Le Havre,NaN,Stade Yves Allainmat,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,False
38157,34.0,Sat,2026-05-16,NaN,Lille,nan,Auxerre,NaN,Decathlon Arena - Stade Pierre-Mauroy,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,False
38158,34.0,Sat,2026-05-16,NaN,Brest,nan,Angers,NaN,Stade Francis-Le Blé,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,False
38159,34.0,Sat,2026-05-16,NaN,Lyon,nan,Lens,NaN,Groupama Stadium,NaN,...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,False
